<h3>Ian Schweer</h3>
<h3>22514022</h3>
<h3>CS 177 - Naive Bayes</h3>

In [6]:
import numpy as np
import matplotlib.pyplot as plt

class params:
	_targets = []
	_data = []
	_classes = []
	_classProbs = []
	def __init__(self, Data, Classes, Targets):
		self._targets = np.array(Targets)
		self._data = np.array(Data)
		self._classes = np.array(Classes)
		self._classProbs = np.array([self.lenfrac(Data[Targets==Classes[0]]) + 1, self.lenfrac(Data[Targets==Classes[1]]) + 1])

	# get frame 
	def lenfrac(self, var):
		return 1. * len(var)
	def params(self, j, i):
		if (j < 0 or j >= self._data.shape[1]):
			raise Exception("j out of bounds",j)
		return self._data[self._targets == i, j]

	def mprobs(self, j, k, i):
		var = self.params(j, i);
		return var[var == k]

	def getclass(self, c):
		return self._classProbs[self._classes==c][0]

	def classprobs(self, c):
		return self.getclass(c) / len(self._data)

	def pofx(self, x):
		# given a vector of x values
		# compute the probability of the value
		# then multiply them all together.
		px = 1.0
		for i, xj in enumerate(x):
			X = self._data[:,i]
			counter = 0
			for y in X:
				counter = counter + (y == xj)
			px = (counter / self.lenfrac(X))
		return px

	def cprobs(self, j, k, i):
		denom = self.getclass(i)
		head = self.lenfrac(self.mprobs(j,k,i))
		# naive bayes assumes conditional independence. So P[X|Y] = P(X)P(Y)
		return (head + .5) / self.getclass(i)

X = np.genfromtxt('data.txt')
Y = np.genfromtxt('labels.txt')

classes=[1,2]
p = params(X, classes, Y)
print "p(C = 1) =",p.getclass(classes[0]) / len(X)
print "p(C = 2) =",p.getclass(classes[1]) / len(X)

for c in classes:
	for i in range(0,X.shape[1]): 
		ps = [p.cprobs(i, 1, c), p.cprobs(i, 2, c)]
		print "P(X =",i,"| C =",c,") [1, 2] = ", ps

#train using only a subset of data.
Xte,Yte = X[1500:], Y[1500:]
predictions = np.array([])
for i in [1500, 50, 10]:
	Xtr,Ytr = (X[0:i], Y[0:i])
	p = params(Xtr, classes, Ytr)
	for d in range(Xte.shape[0]):
		pxji = np.empty(2) # P(Xj|C=i)
		pxji.fill(1.0)
		for ci,c in enumerate(classes):
			temp = 1.0
			for ji,j in enumerate(Xte[d]):
				temp = temp * p.cprobs(ji, j, c)
			pxji[ci] = temp * p.classprobs(c)
		predictions=np.append(predictions, classes[pxji.argmax()])
	print np.mean(predictions.reshape(Yte.shape) != Yte)
	predictions = np.array([])		

p(C = 1) = 0.60617257118
p(C = 2) = 0.394262116931
P(X = 0 | C = 1 ) [1, 2] =  [0.85209752599498023, 0.14790247400501971]
P(X = 1 | C = 1 ) [1, 2] =  [0.90193617784152025, 0.098063822158479738]
P(X = 2 | C = 1 ) [1, 2] =  [0.72266045177482974, 0.27733954822517032]
P(X = 3 | C = 1 ) [1, 2] =  [0.99695231265686624, 0.0030476873431337396]
P(X = 4 | C = 1 ) [1, 2] =  [0.77967013266403729, 0.22032986733596271]
P(X = 5 | C = 1 ) [1, 2] =  [0.88580136249551811, 0.11419863750448189]
P(X = 6 | C = 1 ) [1, 2] =  [0.98440301183219792, 0.01559698816780208]
P(X = 7 | C = 1 ) [1, 2] =  [0.92631767658659014, 0.073682323413409828]
P(X = 8 | C = 1 ) [1, 2] =  [0.92165650770885621, 0.078343492291143776]
P(X = 9 | C = 1 ) [1, 2] =  [0.8295087845105773, 0.17049121548942273]
P(X = 10 | C = 1 ) [1, 2] =  [0.94890641807099319, 0.051093581929006815]
P(X = 11 | C = 1 ) [1, 2] =  [0.5821082825385443, 0.4178917174614557]
P(X = 12 | C = 1 ) [1, 2] =  [0.8807816421656508, 0.11921835783434923]
P(X = 13 | C = 1 ) [1

<b>How does the accuracy vary across experiements?</b>
<br>

<div>Our error rate <p style="display:inline;font-weight:bold">increases</p>. This is because our naive bayes classifer can only use data it has seen too determine it's prediction. As we lower the amount of data feed into the classifer, the less information it has to determine the output of a feature vector is hasn't seen before.</div>

<b>Suppose a classifier randomly guessed the class labels with equal probabilities of 0.5. How accurate would you expect it to be?</b><br>

<div>For this classifier, P(C=i | Xj) = .5, which means we would need a way to choose the class C. Assume that C is binary [Spam, Not Spam], and we always pick a class based off some random method (e.g. random number generator). That means at best, ever item in our dataset happens to be class as our guess, giving us 0% error. That is unlikely. It's also unlikely that we get every single item wrong. Given that, I would assume this classifier would get about <p style="font-weight:bold;display:inline;">50% incorrect</p>, with a 10% fluctuation on each side, depending on the "luck" of our random number generator.</div> 

<b>How accurate would a classifier be that always predicted the class label which occurred most commonly in the training set (i.e., if the majority of examples 1–1500 were class i, it would always report class i regardless of input)?</b>

<div>This classifer could predict as well as the previous classifer, with about a 50% error. However, this classifer is susceptible to bad sampling. An example, what if only 33% of our data is class 1 and that happens to be our training data? Then we would 100% on the testing set, and probably 50% error on the remaining. If this was a rather representative sample, then we would have an error close to ratio of the minority element in the dataset. On average, I suspect this classifer to get roughly <p style="font-weight:bold;display:inline;">50%</p> incorrect.</div>

<b>How do these two simple strategies compare to the performance of your system measured in parts 1-3?</b>

<div>This two classifers are definietly easier to implement then the one I built. However, they would do really bad in practice. The reason for this is neither classifer learns from our dataset. Our classifer attempts to understand the class labeling from a cooresponding feature vector.</div>

<b>Write a third function which returns the probability of the class label C being spam and non-spam. Compute these probabilities for the first 20 examples in the data file. You do not have to turn in code for this function, just the table of values. You will probably want to use the code you have already written for nbayes predict.m as a template and modify it slightly to return the class probabilities rather than just the most likely class.</b>

In [7]:
Xte,Yte = X[20:], Y[20:]
predictions = np.array([])
for i in [20]:
	Xtr,Ytr = (X[0:i], Y[0:i])
	p = params(Xtr, classes, Ytr)
	for d in range(Xtr.shape[0]):
		pxji = np.empty(2) # P(Xj|C=i)
		pxji.fill(1.0)
		for ci,c in enumerate(classes):
			temp = 1.0
			for ji,j in enumerate(Xtr[d]):
				temp = temp * p.cprobs(ji, j, c)
			pxji[ci] = (temp * p.classprobs(c)) / p.pofx(Xtr[d])
		print pxji


[  1.41543637e-13   1.51357769e-07]
[  1.17329313e-08   3.75416247e-13]
[  9.95858565e-09   2.48213923e-08]
[  5.03947933e-14   1.57412080e-08]
[  3.55752516e-08   8.49358025e-15]
[  8.20289353e-08   1.79510398e-10]
[  2.41290270e-12   6.31817258e-10]
[  6.50762886e-07   2.19680557e-10]
[  6.04516570e-19   2.48213923e-08]
[  4.34270834e-08   3.35982028e-11]
[  2.44451735e-18   9.20677170e-29]
[  2.91144988e-12   2.90042214e-26]
[  3.20521467e-14   2.03283244e-14]
[  5.06840779e-14   5.38748322e-26]
[  2.07990511e-10   2.24627985e-11]
[  2.60305155e-08   2.19680557e-10]
[  3.44521528e-07   1.29223857e-11]
[  6.84459274e-28   2.99085093e-14]
[  2.86841849e-10   1.87708124e-12]
[  3.62629200e-17   1.81342137e-09]
